In [1]:
import pandas as pd
from copy import deepcopy

doubleSpaces = [4,6,8]
doubleIndices = [11,12,13]

class Board():
    # creates a standard board
    # first position is not usable
    board = None
    prevNumMoved = None
    totalMoves = 0
    moves = []

    def __init__(self):
        self.board = [0 for i in range(14)]
        for i in range(2,10): self.board[i] = i
        self.board[0] = -1
        self.board[10] = 1
    
    def copy(self):
        c = Board()
        c.board = deepcopy(self.board)
        c.prevNumMoved = self.prevNumMoved
        c.totalMoves = self.totalMoves
        c.moves = self.moves
        return c

    def display(self):
        a = pd.Series(self.board[1:11])
        b = pd.Series([' ',' ',' ', self.board[11], ' ', self.board[12], ' ', self.board[13], ' ', ' '])
        df = pd.DataFrame([b,a])
        return df.to_string(index=False, header=False)
    
    def isSolved(self):
        return self.board == [0,1,2,3,4,5,6,7,8,9,0,0,0,0]

# FOR CHECKDOWN AND CHECKUP:
# i :  
    
    def checkDown(self, i):
        try:
            x = self.board.index(i)
            if self.board.index(i) in doubleIndices:
                if self.board[ doubleSpaces[ doubleIndices.index(x) ] ] == 0: 
                    return (True, doubleSpaces[ doubleIndices.index(x) ] )
        except:
            return (False, None)

        return (False, None)

    def checkUp(self, i):
        try:
            x = self.board.index(i)
            if self.board.index(i) in doubleSpaces:
                if self.board[ doubleIndices[ doubleSpaces.index(x) ] ] == 0: 
                    return (True, doubleIndices[ doubleSpaces.index(x) ] )
        except:
            return (False, None)

        return (False, None)

# FOR CHECKLEFT AND CHECKRIGHT:
# i : NUMBER whose left/right position will be checked. Will return a tuple: (is valid move, index checked)
# if i is in doubleIndices, it will automatically return (False, None)

    def checkLeft(self, i):
        try:
            x = self.board.index(i)
            if x-1 not in doubleIndices:
                if self.board[x-1] == 0: return (True, x-1)
        except:
            return (False, None)

        return (False, None)

    def checkRight(self, i):
        try:
            x = self.board.index(i)
            if x+1 not in doubleIndices:
                if self.board[x+1] == 0: return (True, x+1)
        except:
            return (False, None)
        return (False, None)


# i : number whose possible moves will be returned
# finalset keeps track of all SEEN valid moves from current position
# currentset keeps track of unexplored states

    def possibleMoves(self, i):
        finalset = [self.board.index(i)]
        currentset = []

        up = self.checkUp(i)
        down = self.checkDown(i) 
        left = self.checkLeft(i) 
        right = self.checkRight(i)  

        if up[0]: finalset.append(up[1]); currentset.append(up[1]); # print('here1')
        if down[0]: finalset.append(down[1]); currentset.append(down[1]); # print('here2')
        if left[0]: finalset.append(left[1]); currentset.append(left[1]); # print('here3')
        if right[0]: finalset.append(right[1]); currentset.append(right[1]); # print('here4')
        
        while currentset:
            temp = self.copy()
            temp.move( (i, currentset[len(currentset) - 1]) )
            currentset = currentset[:-1]

            up = temp.checkUp(i)
            down = temp.checkDown(i)
            left = temp.checkLeft(i)
            right = temp.checkRight(i)    

            if up[0]:
                if not (up[1] in finalset): 
                    # print('here1')
                    finalset.append(up[1])
                    currentset.append(up[1])
            if down[0]:
                if not (down[1] in finalset): 
                    # print('here2')
                    finalset.append(down[1])
                    currentset.append(down[1])
            if left[0]:
                if not (left[1] in finalset): 
                    # print('here3')
                    finalset.append(left[1])
                    currentset.append(left[1])
            if right[0]:
                if not (right[1] in finalset): 
                    # print('here4')
                    finalset.append(right[1])
                    currentset.append(right[1])

        return finalset[1:]
        

    # executes a valid move
    # move - tuple (number, new position)
    def move(self, move):
        self.board[self.board.index(move[0])] = 0
        self.board[move[1]] = move[0]
        self.moves.append(move)
        if self.prevNumMoved == move[0]:
            return
        else:
            self.prevNumMoved = move[0]
            self.totalMoves += 1


    # gets a list of all possible game boards that can result from the current in one move (does not add moves that have already been seen)
    def expand(self, seenNodes):
        nodes = []

        runninglist = []
        for i in range(1,10):
            runninglist.append([i, self.possibleMoves(i)])

        for item in runninglist:
            if item[1] == []:
                continue
            else: 
                for position in item[1]:
                    temp = self.copy()
                    temp.move((item[0], position))
                    nodes.append(temp)
                    alreadySeen = False
                    for seenNode in seenNodes:
                        if temp.board == seenNode.board: alreadySeen = True
                    if alreadySeen == False: nodes.append(temp)

        return nodes

    # This heuristic will only check to see how far 1 is from its position
    def heuristicA(self):
        distance = 0

        x = self.board.index(1)
        if x in doubleIndices:
            distance = abs(doubleSpaces[doubleIndices.index(x)])
        else:
            distance = abs(x - 1)
        return distance + self.totalMoves
    
    # This heuristic will sum the distances of each number from its respective position
    def heuristicB(self):
        distance = 0

        for i in range(1,10):
            x = self.board.index(i)
            if x in doubleIndices:
                distance += abs(doubleSpaces[doubleIndices.index(x)] - i + 1)
            else: 
                distance += abs(x - i)
        return distance + self.totalMoves


In [2]:
def UniformSearch(b):
    ALLNODES = [b]
    nodes = [ALLNODES[0]]

    i = 0
    while nodes:
        i += 1
        currentnode = nodes[0]
        nodes = nodes[1:]

        if currentnode.isSolved(): return (currentnode, i)

        expantion = currentnode.expand(ALLNODES)
        for newnode in expantion:
            nodes.append(newnode)
            ALLNODES.append(newnode)
            
    return (-1, -1)

def aStarHeuristicA(b):
    ALLNODES = [b]
    nodes = [ALLNODES[0]]

    i = 0
    while nodes:
        i += 1
        nodes = sorted(nodes, key=lambda x: x.heuristicA())

        currentnode = nodes[0]
        nodes = nodes[1:]

        if currentnode.isSolved(): return (currentnode, i)

        expantion = currentnode.expand(ALLNODES)
        for newnode in expantion:
            nodes.append(newnode)
            ALLNODES.append(newnode)
            
    return (-1, -1)

def aStarHeuristicB(b):
    ALLNODES = [b]
    nodes = [ALLNODES[0]]

    i = 0
    while nodes:
        i += 1
        nodes = sorted(nodes, key=lambda x: x.heuristicB())

        currentnode = nodes[0]
        nodes = nodes[1:]

        if currentnode.isSolved(): return (currentnode, i)

        expantion = currentnode.expand(ALLNODES)
        for newnode in expantion:
            nodes.append(newnode)
            ALLNODES.append(newnode)
            
    return -1


In [3]:
beginner = Board(); beginner.board = [0,0,1,2,3,5,6,7,8,9,0,4,0,0]
intermediate = Board(); intermediate.board = [0,2,3,4,0,0,0,5,6,8,9,1,0,0,7]
print(beginner.display(), '\n')
print(intermediate.display(), '\n')

      4   0   0    
0 1 2 3 5 6 7 8 9 0 

      1   0   0    
2 3 4 0 0 0 5 6 8 9 



In [6]:
results = aStarHeuristicA(beginner)
print(results[0].display(), '\n' ,results[1])

In [4]:
results = aStarHeuristicB(beginner)
print(results[0].display(), '\n' ,results[1])

      0   0   0    
1 2 3 4 5 6 7 8 9 0 
 11


In [5]:
results = UniformSearch(beginner)
print(results[0].display(), '\n' ,results[1])

      0   0   0    
1 2 3 4 5 6 7 8 9 0 
 1527


In [5]:
results = aStarHeuristicB(intermediate)
print(results[0].display(), '\n' ,results[1])

In [ ]:
results = UniformSearch(intermediate)
print(results[0].display(), '\n' ,results[1])

In [ ]:
results = aStarHeuristicA(intermediate)
print(results[0].display(), '\n' ,results[1])